<a href="https://colab.research.google.com/github/Buchiexplores/Big_Data_Training/blob/master/Assignment5/Scripts/Spotipy_Data_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Abuchi Okeke
#10/04/2020
#Python script to integrate Spotify Web APIs and extract data based on playlists using spotipy lib.

#pip install spotipy

In [ ]:
#import libs
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

#Users credentials
CLIENT_ID = 'fda982ead35c41c2b8b3e2763e7a0304'
CLIENT_SECRET = '7de8b0a3d1ca447f8db9d509ce8e5f1d'

#generate token
token = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
cache_token = token.get_access_token()
sp = spotipy.Spotify(cache_token)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  if sys.path[0] == '':


In [ ]:
#test integration
user_id = "tjmaml1jx29bsbqeazj9vir0o"
playlist_id = "5KbTzqKBqxQRD8OBtJTZrS"
sp.user_playlist_tracks(user_id,  playlist_id)

In [ ]:

#functio to extract data from single playlist
def analyze_playlist(creator, playlist_id):
    
    # Create empty dataframe
    playlist_features_list = ["artist","album","track_name",  "track_id","danceability","energy","key","loudness","mode", "speechiness","instrumentalness","liveness","valence","tempo", "duration_ms","time_signature"]
    
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    # Loop through every track in the playlist, extract features and append the features to the playlist df
    
    playlist = sp.user_playlist_tracks(creator, playlist_id)["items"]
    for track in playlist:
        # Create empty dict
        playlist_features = {}
        # Get metadata
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        
        # Get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[4:]:
            playlist_features[feature] = audio_features[feature]
        
        # Concat the dfs
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
        
    return playlist_df

In [ ]:
#clean data and export to .csv using panda
import pandas as pd

#extract data into .csv file
df = analyze_playlist("tjmaml1jx29bsbqeazj9vir0o", "5KbTzqKBqxQRD8OBtJTZrS")
# csv
df.to_csv("spotify_output.csv", index = False)
# excel
#df.to_excel("dataframe.xlsx", index = False)

In [ ]:
#function to extract from multiple playlists
def analyze_playlist_dict(playlist_dict):
    
    # Loop through every playlist in the dict and analyze it
    for i, (key, val) in enumerate(playlist_dict.items()):
        playlist_df = analyze_playlist(*val)
        # Add a playlist column so that we can see which playlist a track belongs too
        playlist_df["playlist"] = key
        # Create or concat df
        if i == 0:
            playlist_dict_df = playlist_df
        else:
            playlist_dict_df = pd.concat([playlist_dict_df, playlist_df], ignore_index = True)
            
    return playlist_dict_df

In [ ]:
#create multiple playlist ids Dict
playlist_dict = {
 "warm_fuzzy_feeling" : ("tjmaml1jx29bsbqeazj9vir0o", "37i9dQZF1DX5IDTimEWoTd"), 
 "love_songs_heart" : ("tjmaml1jx29bsbqeazj9vir0o", "5KbTzqKBqxQRD8OBtJTZrS"),
 "romance_songs" : ("tjmaml1jx29bsbqeazj9vir0o", "7sAUK3XK8NHH1s5vGcTBkF"),
 "khelani_complete": ('tjmaml1jx29bsbqeazj9vir0o', '4R0BZVh27NUJhHGLNitU08')
}

In [ ]:
#extract data from multiple playlist
multiple_playlist_df = analyze_playlist_dict(playlist_dict)
multiple_playlist_df["playlist"].value_counts()
multiple_playlist_df.to_csv (r'/content/Spotify_multiple_playlist.csv', index = None)

In [ ]:
#test spotipy in-built modules for audio analysis 
#track_id =""
#sp.audio_analysis(track_id)
#sp.audio_features(track_id)
